# Chapter 01 — Python & Linear Algebra for Finite Elements

**Duration:** ~3 hours  
**Focus:** Essential Python and matrix operations needed for Finite Element Method (FEM).

### Learning outcomes
By the end, you will be able to:
- Set up and use Python with NumPy for vector/matrix math.
- Assemble and solve linear systems $A\,x=b$.
- Understand dense vs. sparse storage and when each is appropriate.
- Plot basic results for quick checks.
- Implement a tiny 1D bar FEM from scratch (stiffness assembly, BCs, solve).

## 0. Setup

In [ ]:
import sys, math
print(sys.version)
try:
    import numpy as np
    import matplotlib.pyplot as plt
    print('NumPy:', np.__version__)
except Exception as e:
    raise SystemExit('This notebook requires NumPy and Matplotlib.')

# Optional: SciPy for sparse matrices (not strictly required for small examples)
try:
    import scipy
    import scipy.sparse as sp
    import scipy.sparse.linalg as spla
    HAS_SCIPY = True
    print('SciPy:', scipy.__version__)
except Exception:
    HAS_SCIPY = False
    print('SciPy not available — examples will fall back to dense matrices for small N.')

## 1. Python quick primer (for scientific computing)

In [ ]:
# Basic types & containers
x = 3.0                # float
i = 5                  # int
v_list = [1, 2, 3]     # Python list
t_tuple = (4, 5, 6)    # tuple (immutable)
print(x, type(x)); print(i, type(i)); print(v_list, type(v_list)); print(t_tuple, type(t_tuple))

# Vectorization with NumPy
import numpy as np
v = np.array([1., 2., 3.])
w = np.array([4., 5., 6.])
print('v + w =', v + w)
print('v · w =', v @ w)         # dot product
print('‖v‖2  =', np.linalg.norm(v))

## 2. Matrix creation & operations

In [ ]:
A = np.array([[2., -1., 0.],
              [-1., 2., -1.],
              [0., -1., 2.]])
b = np.array([1., 0., 1.])
print('A =\n', A)
print('b =', b)

# Solve Ax=b
x = np.linalg.solve(A, b)
print('Solution x =', x)

# Check residual r = Ax - b
r = A @ x - b
print('Residual ‖r‖2 =', np.linalg.norm(r))

# Useful helpers
print('A.T =\n', A.T)
print('Symmetric?', np.allclose(A, A.T))
print('Condition number κ(A) =', np.linalg.cond(A))

## 3. Indexing, slicing, broadcasting

In [ ]:
u = np.arange(10.)  # [0,1,...,9]
print('u      =', u)
print('u[2:7] =', u[2:7])
print('u[::2] =', u[::2])

# Boolean masking
mask = (u % 2 == 0)
print('even mask   =', mask)
print('u[mask]     =', u[mask])

# Broadcasting example: add row vector to each row
M = np.ones((3,4))
row = np.array([0.,1.,2.,3.])
print('M + row =\n', M + row)

## 4. Dense vs. sparse matrices

In [ ]:
def tridiag(n, a=-1.0, b=2.0, c=-1.0, sparse=False):
    if sparse and HAS_SCIPY:
        diags = [a*np.ones(n-1), b*np.ones(n), c*np.ones(n-1)]
        return sp.diags(diags, offsets=[-1,0,1], format='csr')
    else:
        A = np.zeros((n,n))
        for i in range(n):
            A[i,i] = b
            if i>0:   A[i,i-1] = a
            if i<n-1: A[i,i+1] = c
        return A

n = 10
A_dense = tridiag(n, sparse=False)
print('Dense example shape:', A_dense.shape)
if HAS_SCIPY:
    A_sparse = tridiag(n, sparse=True)
    print('Sparse example (CSR):', A_sparse.shape, '| nnz =', A_sparse.nnz)
else:
    print('SciPy not available: skipping sparse demo')

## 5. Mini FEM: 1D bar with 2-node linear elements

We solve $-(EA\,u')' = f$ on $x\in[0,L]$ with $u(0)=0$ and a traction $t$ at $x=L$.
Assembly steps:
1. Discretize the domain into $N_e$ elements (uniform).
2. For each element, local stiffness $k_e = \dfrac{EA}{h} \begin{bmatrix} 1 & -1\\ -1 & 1\end{bmatrix}$ and load $f_e$ (here taken as constant body force or zero).
3. Scatter-add into global $K$ and $f$.
4. Apply Dirichlet BC at node 0 (clamped): remove row/col or enforce with penalty.
5. Solve $K\,u=f$ and post-process strain $\varepsilon$ and stress $\sigma=E\,\varepsilon$.

In [ ]:
def fem_1d_bar(E=210e9, A=1e-4, L=1.0, Ne=10, body=0.0, traction=1000.0):
    import numpy as np
    nn = Ne + 1
    h  = L / Ne
    K  = np.zeros((nn, nn))
    f  = np.zeros(nn)

    ke = (E*A/h) * np.array([[1., -1.],
                             [-1., 1.]])
    fe_body = (body*h/2.0) * np.array([1., 1.])

    # Assembly
    for e in range(Ne):
        n1, n2 = e, e+1
        dofs = [n1, n2]
        K[np.ix_(dofs, dofs)] += ke
        f[dofs] += fe_body

    # Traction at x=L (Neumann)
    f[-1] += traction

    # Dirichlet BC: u(0)=0 (remove row/col)
    K = K[1:,1:]
    f = f[1:]

    u_free = np.linalg.solve(K, f)
    u = np.concatenate(([0.0], u_free))
    x = np.linspace(0., L, nn)

    # Element strains & stresses (constant per element for linear 2-node)
    strain = np.diff(u)/h
    stress = E*strain
    x_mid  = x[:-1] + 0.5*h
    return x, u, x_mid, strain, stress

x, u, x_mid, eps, sig = fem_1d_bar(Ne=20)
print('Max displacement:', u.max())
print('Max stress      :', abs(sig).max())

### Plotting displacement and stress

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(x, u, marker='o')
plt.xlabel('x [m]'); plt.ylabel('displacement u [m]'); plt.title('1D bar — displacement')
plt.show()

plt.figure()
plt.plot(x_mid, sig, marker='s')
plt.xlabel('x [m]'); plt.ylabel('stress σ [Pa]'); plt.title('1D bar — stress (piecewise-constant)')
plt.show()

## 6. Exercises (in-class / homework)

1. **Vector & matrix warm‑up:**
   - Create a random SPD matrix `A` (e.g., `A = M.T@M + αI`) and a vector `b`, then solve `Ax=b`. Report the residual norm.
2. **Conditioning:**
   - Build `A=tridiag(n)` for `n=20,40,80`. Compute `cond(A)` and discuss the trend.
3. **Sparse practice (if SciPy available):**
   - Use CSR to build a 1D Poisson matrix with Dirichlet BCs and solve using `spla.spsolve`.
4. **FEM extension:**
   - Modify `fem_1d_bar` to include a uniform body force density `body≠0`.
   - Change boundary conditions to `u(0)=0` and specified displacement `u(L)=u0` (both Dirichlet) using row/column elimination or a penalty method.
5. **Verification:**
   - For `body=0`, compare `u(x)` against the analytical solution for a bar under end load `t`: `u(x)=t x/(E A)`.


---
**Instructor note:** You can add solution cells below as needed or distribute a separate solution key.